In [ ]:
import logging
import json
import numpy as np
import pandas as pd

from apsopt.core.optimizer import Optimizer, get_generator_class
from apsopt.core.evaluators import FunctionEvaluator
from apsopt.util.pydantic.options import AlgorithmOptions
from apsopt.util.pydantic.vocs import GVOCS

from opt_funcs_nsls import make_dummy_eval

logger = logging.getLogger(__name__)

SETUP

In [ ]:
# Load precomputed data

MODE = '6and3'
with open(f'lattice/data_{MODE}.json', 'r') as f:
    data_dict = json.load(f)
groups = data_dict['groups']
groups_direct = data_dict['groups_direct']
all_groups = {**groups, **groups_direct}
indices = data_dict['indices']
null_knobs = data_dict['null_knobs']
dfknobs = pd.from_json(f'lattice/dfknobs_{MODE}.json')


# For direct groups, means +-15A
variables = {k:[-15,15] for k in null_knobs.keys()}

objectives = {'LT':'maximize', 'EFF':'maximize'}

reference_point = {'LT':0.7, # hours
                   'EFF':0.2 # fraction injected
                   }

# TODO: set from lattice save???
initial_variable_values = {'x0':0.1, 'x1':0.1}

gvocs = GVOCS(variables=variables,
              variables_active=variables,
              objectives=objectives,
              objectives_active=objectives)

In [ ]:
eval_f = make_dummy_eval(gvocs)
ev = FunctionEvaluator(script=eval_f)

generator_name = 'aps_mobo'
gc = get_generator_class(generator_name)

gen = gc(gvocs=gvocs)
gen.numerical_optimizer.n_raw_samples = 512
gen.numerical_optimizer.n_restarts = 10
gen.gp_constructor.use_low_noise_prior = False
gen.gp_constructor.use_high_noise_prior = False
#gen.max_travel_distances = [0.1] * len(gvocs.variables_active)
gen.reference_point = reference_point

In [ ]:
options = AlgorithmOptions(store_history=True, dump_every_step=True)
opt = Optimizer(options=options,
                generator=gen,
                evaluator=ev,
                gvocs=gvocs)

Sample at initial values (i.e. standard lattice)

In [ ]:
opt.generate_at(initial_variable_values)
opt.evaluate_data()

In [ ]:
opt.data

Sample close to initial point

In [ ]:
mt = {'x0':0.05, 'x1':0.05}
opt.generate_random_around_point(n=1, point=initial_variable_values, spans=mt, use_normalized_distances=True)
opt.evaluate_data()

In [ ]:
opt.data

Run optimizer

In [ ]:
import warnings
#warnings.filterwarnings("ignore", message="Unable to find non-zero acquisition function values")

for i in range(20):
    opt.step()

In [ ]:
opt.state_to_json()

In [ ]:
opt.data.tail()

In [ ]:
import matplotlib.pyplot as plt

hvs = []
for i in range(1,len(opt.data)):
    hv = opt.generator.calculate_hypervolume_at(i)
    hvs.append(hv)

fig,ax = plt.subplots(1,2,squeeze=False,figsize=(12,6))
df = opt.data
ol = ['EFF', 'LT']
l = ax[0,0].scatter(df.loc[:,ol[0]], df.loc[:,ol[1]],10,marker='o',c=range(len(df)),vmax=len(df))
ax[0,0].scatter(df.loc[:1,ol[0]], df.loc[:1,ol[1]], 20,marker='o',c='red',label='initial point')
ax[0,0].scatter(gen.reference_point[ol[0]], gen.reference_point[ol[1]],40,marker='*',c='black',label='reference point')
ax[0,0].set_xlabel(ol[0])
ax[0,0].set_ylabel(ol[1])
fig.colorbar(l)
ax[0,0].legend()

ax[0,1].plot(hvs,marker='o')
ax[0,1].set_xlabel('Step #')
ax[0,1].set_ylabel('Hypervolume')

In [ ]:
# This will save optimizer state
opt.state_to_json(f'exp_mobo_{MODE}_v1_dump.json')